# 1. Imports and chart setup

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
sns.set_style("whitegrid")
sns.set(rc={'figure.figsize':(20,8)})
sns.set(font_scale=1.5)

# 2. Data Cleaning

In [ ]:
# importing line_items data and changing column names to lowercase
items = pd.read_csv('line_items.csv')
items.columns = items.columns.str.lower()

In [ ]:
# data cleaning in items df
items['order_id'] = items['order_id'].astype(str) # type to string
items['print_provider_id'] = items['print_provider_id'].fillna(0).astype(int) # type to integer, NULLs changed to 0
items['quantity'] = items['quantity'].astype(int) # type to integer
items["reprint_flag"].fillna(False,inplace=True) # type to boolean, NULLs changed to False

In [ ]:
# items.head()

In [ ]:
# importing orders data and changing column names to lowercase
orders = pd.read_csv('orders.csv')
orders.columns = orders.columns.str.lower()

In [ ]:
# data cleaning in orders df
orders['order_id'] = orders['order_id'].astype(str) # type to string
orders['order_dt'] = pd.to_datetime(orders['order_dt']) # type to datetime
orders['fulfilled_dt'] = pd.to_datetime(orders['fulfilled_dt']) # type to datetime
orders['merchant_registered_dt'] = pd.to_datetime(orders['merchant_registered_dt']) # type to datetime
orders['shipment_delivered_at'] = pd.to_datetime(orders['shipment_delivered_at']) # type to datetime

In [ ]:
# orders.head()

# 3. Merchants

### What characteristics do the most successful merchants share?

In [ ]:
# creating merchants_data df used next for grouping
merchants_data = orders[['merchant_id', 'order_id', 'shop_id', 'address_to_country', 'address_to_region',
                         'sales_channel_type_id', 'total_cost']]
merchants_data = merchants_data.drop_duplicates()

In [ ]:
# merchants_data.head()

### 3.1 Boxplots for top 50 merchants characteristics

In [ ]:
# grouping data for each merchant in merchants_group df and selecting top 50 merchants per total sales value
# df will be used next for creating boxplots
merchants_group = merchants_data.groupby('merchant_id').agg(
            orders_count = ('order_id', 'nunique'),
            total_sales = ('total_cost', sum),
            avg_sale = ('total_cost', 'mean'),
            mdn_sale = ('total_cost', 'median'),
            shops_count = ('shop_id', 'nunique'),
            countries_count = ('address_to_country', 'nunique'),
            regions_count = ('address_to_region', 'nunique'),
            channels_count = ('sales_channel_type_id', 'nunique')
)
merchants_group['avg_sale'] = round(merchants_group['avg_sale'], 2)
merchants_group['mdn_sale'] = round(merchants_group['mdn_sale'], 2)
merchants_group.reset_index(inplace=True)
merchants_group = merchants_group.sort_values(by='total_sales', ascending=False).head(50)

In [ ]:
# merchants_group.head()

In [ ]:
# creating boxplots for 4 different metrics as they can be shown on one chart

sns.boxplot(data=merchants_group[["shops_count", "avg_sale", "mdn_sale", "countries_count"]], orient='v', showfliers=False)
plt.show()

In [ ]:
# creating boxplot for number of regions

sns.boxplot(data=merchants_group[['regions_count']], orient='v', showfliers=False)
plt.show()

### 3.2 Main countries and regions for top 50 merchants

In [ ]:
# selecting for each merchant country where they sell most products. Next preparing pie chart to present data
merchants_countries = merchants_data[['merchant_id', 'address_to_country', 'order_id']].groupby(
                            ['merchant_id', 'address_to_country']).count()
merchants_countries.reset_index(inplace=True)
merchants_countries = merchants_countries.merge(merchants_group['merchant_id'], on='merchant_id')

merchants_filter = merchants_countries[['merchant_id', 'order_id']].groupby('merchant_id').max()
merchants_filter.reset_index(inplace=True)

merchants_countries = merchants_countries.merge(merchants_filter, on=('merchant_id', 'order_id'))

countries = merchants_countries[['address_to_country', 'order_id']].groupby('address_to_country').count()
countries.reset_index(inplace=True)
countries.columns = ['country', 'merchants']
countries.sort_values(by='merchants', ascending=False, inplace=True)

In [ ]:
# creating pie chart for countries

colors = sns.color_palette('pastel')[0:len(countries)]

plt.pie(countries['merchants'], labels = countries['country'], colors = colors, autopct='%.0f%%')
plt.title('COUNTRIES', size=20)
plt.show()

In [ ]:
# selecting for each merchant region where they sell most products. Next preparing pie chart to present data
merchants_regions = merchants_data[['merchant_id', 'address_to_region', 'order_id']].groupby(
                            ['merchant_id', 'address_to_region']).count()
merchants_regions.reset_index(inplace=True)
merchants_regions = merchants_regions.merge(merchants_group['merchant_id'], on='merchant_id')

merchants_filter = merchants_regions[['merchant_id', 'order_id']].groupby('merchant_id').max()
merchants_filter.reset_index(inplace=True)

merchants_regions = merchants_regions.merge(merchants_filter, on=('merchant_id', 'order_id'))

regions = merchants_regions[['address_to_region', 'order_id']].groupby('address_to_region').count()
regions.reset_index(inplace=True)
regions.columns = ['region', 'merchants']
regions.sort_values(by='merchants', ascending=False, inplace=True)

In [ ]:
# creating pie chart for regions

colors = sns.color_palette('pastel')[0:len(regions)]

plt.pie(regions['merchants'], labels = regions['region'], colors = colors, autopct='%.0f%%')
plt.title('REGIONS', size=20)
plt.show()

# 4. Shipping Carriers

### What are the top two shipping carriers? Why should or shouldn’t we try to use those two for all orders?

In [ ]:
# creating carriers_data df used next for grouping
carriers_data = orders[['shipment_carrier', 'order_id', 'total_shipping', 'fulfilled_dt', 
                        'shipment_delivered_at', 'address_to_country', 'address_to_region']]
carriers_data = carriers_data[carriers_data['shipment_carrier'].notnull()]
carriers_data = carriers_data.drop_duplicates()
# calculating shipping time for each order
carriers_data['shipping_time'] = carriers_data['shipment_delivered_at'] - carriers_data['fulfilled_dt']

In [ ]:
# carriers_data.head()

In [ ]:
# grouping data for each carrier in carriers_group df and selecting all carriers that delivered over 100 orders
carriers_group = carriers_data.groupby('shipment_carrier').agg(
            orders_count = ('order_id', 'nunique'),
            total_shipping_costs = ('total_shipping', sum),
            avg_ship_time = ('shipping_time', 'mean'),
            mdn_ship_time = ('shipping_time', 'median'),
            avg_ship_cost = ('total_shipping', 'mean'),
            mdn_ship_cost = ('total_shipping', 'median')
)
carriers_group['avg_ship_cost'] = round(carriers_group['avg_ship_cost'], 2)
carriers_group['mdn_ship_cost'] = round(carriers_group['mdn_ship_cost'], 2)
carriers_group['avg_ship_time'] = carriers_group['avg_ship_time'].round('H') # rounding to hours
carriers_group['mdn_ship_time'] = carriers_group['mdn_ship_time'].round('H') # rounding to hours
carriers_group.reset_index(inplace=True)
carriers_group = carriers_group.loc[carriers_group['orders_count']>100].sort_values(by='orders_count', ascending=False)
carriers_group.reset_index(inplace=True)

In [ ]:
# carriers_group.head()

### 4.1 Countries and regions comparison

In [ ]:
# selecting top carriers and counting how many orders they delivered to each country
top_carriers = carriers_group['shipment_carrier'].head(2)

top_carriers_countries = carriers_data[['shipment_carrier', 'address_to_country', 'order_id']]\
        .merge(top_carriers, on='shipment_carrier').groupby(['shipment_carrier', 'address_to_country']).nunique()
top_carriers_countries.reset_index(inplace=True)
top_carriers_orders = carriers_group[['shipment_carrier', 'orders_count']].head(2)
top_carriers_countries = top_carriers_countries.merge(top_carriers_orders, on='shipment_carrier')
top_carriers_countries.columns = ['shipment_carrier', 'country', 'orders_to_country', 'all_orders']
# calculating % share of orders delivered to particular country among all orders delivered by carrier
top_carriers_countries['country_share'] = top_carriers_countries['country'] + ' (' + (round(\
            (top_carriers_countries['orders_to_country'] / top_carriers_countries['all_orders']) * 100, 2)).astype(str) + '%)'

In [ ]:
# top_carriers_countries.head()

In [ ]:
# creating bar charts for each top carrier

for carrier in top_carriers:
    
    chart_data = top_carriers_countries.loc[top_carriers_countries['shipment_carrier']==carrier]\
                    .sort_values(by='orders_to_country', ascending=False)
    
    sns.barplot(x = 'country_share',
            y = 'orders_to_country',
            data = chart_data)
    
    plt.title(carrier, size=30)
    plt.xticks(rotation=-90)
    
    plt.show()

In [ ]:
# preparing orders df without top 2 carriers
orders_wo_top_carriers = orders[~orders['shipment_carrier'].isin(top_carriers)]

In [ ]:
# creating countries_orders df to check how many orders were delivered to each country by carriers other than top 2
countries_orders = orders_wo_top_carriers[['address_to_country', 'order_id']].groupby('address_to_country').nunique()
countries_orders.reset_index(inplace=True)
countries_orders.columns = ['country', 'orders_to_country']
all_orders = len(orders_wo_top_carriers['order_id'].unique())
countries_orders['country_share'] = countries_orders['country'] + ' (' + (round(\
            (countries_orders['orders_to_country'] / all_orders) * 100, 2)).astype(str) + '%)'
countries_orders = countries_orders.loc[countries_orders['orders_to_country']>10]

In [ ]:
# creating bar chart for country deliver shares without top carriers

sns.barplot(x = 'country_share',
            y = 'orders_to_country',
            data = countries_orders.sort_values(by='orders_to_country', ascending=False))

plt.title('Orders delivered by other carriers', size=30)
plt.xticks(rotation=-90)
plt.show()

### 4.2 Other metrics comparison

In [ ]:
# checking how many carriers had better and worse results in each category compared to 2 top carriers

carriers_group_wo_top_carriers = carriers_group[~carriers_group['shipment_carrier'].isin(top_carriers)]

metrics = ('avg_ship_time', 'mdn_ship_time', 'avg_ship_cost', 'mdn_ship_cost')

for i in range(0,2):

    for metric in metrics:

        carriers_better = len(carriers_group_wo_top_carriers\
                              .loc[carriers_group_wo_top_carriers[metric]<carriers_group[metric][i]])

        carriers_worse = len(carriers_group_wo_top_carriers\
                              .loc[carriers_group_wo_top_carriers[metric]>carriers_group[metric][i]])
        
        chc = 'carrier has' if carriers_better==1 else 'carriers have'

        print(f"{carriers_better} {chc} better {metric} than {carriers_group['shipment_carrier'][i]},\n\
                    {carriers_worse} have worse.")

# 5. Print Providers

### Which are the two best and worst Print Providers and why?

In [ ]:
# creating df for print provider's orders, with information about destination and print time
providers = items[['print_provider_id', 'order_id']].drop_duplicates()
orders_providers = orders.merge(providers.loc[providers['print_provider_id']!=0], on='order_id')
orders_providers = orders_providers[['print_provider_id', 'order_id', 'address_to_country', 'address_to_region',
                                     'order_dt', 'fulfilled_dt']]
orders_providers['print_time'] = orders_providers['fulfilled_dt'] - orders_providers['order_dt']

In [ ]:
# orders_providers.head()

In [ ]:
# creating df containing important metrics grouped on print provider level

# calculating how many reprints each provider had to make
reprints = items[['print_provider_id', 'quantity']].loc[items['reprint_flag']==True].groupby('print_provider_id').sum()
reprints.reset_index(inplace=True)
reprints.columns = ['print_provider_id', 'reprints']

# caclulating metrics on provider level
providers_group = orders_providers.groupby('print_provider_id').agg(
            orders_count = ('order_id', 'nunique'),
            avg_print_time = ('print_time', 'mean'),
            mdn_print_time = ('print_time', 'median'),
            countries_count = ('address_to_country', 'nunique'),
            regions_count = ('address_to_region', 'nunique')
)
providers_group['avg_print_time'] = providers_group['avg_print_time'].round('H')
providers_group['mdn_print_time'] = providers_group['mdn_print_time'].round('H')

# selecting only print providers that completed more than 20 orders
providers_group = providers_group.loc[providers_group['orders_count']>20]
providers_group.reset_index(inplace=True)

# calculating medians and print percentage for the whole group to compare with individual results
providers_group = providers_group.merge(reprints, how="left", on='print_provider_id')
providers_group['reprint_perc'] = round((providers_group['reprints'] / providers_group['orders_count'])*100, 2)
providers_group['all_mdn_orders'] = providers_group['orders_count'].median().astype(int)
providers_group['all_mdn_countries'] = providers_group['countries_count'].median().astype(int)
providers_group['all_mdn_regions'] = providers_group['regions_count'].median().astype(int)
providers_group['all_mdn_print_time'] = orders_providers['print_time'].median().round('H')
providers_group['all_reprints_perc'] = round((reprints['reprints'].sum() / items['quantity'].sum())*100, 2)

In [ ]:
# providers_group.sort_values(by='orders_count', ascending=False).head()

In [ ]:
# creating df with print providers ranks in each category. Categories to be checked:
# - number of orders
# - median print time
# - number of countries
# - number of regions
# - reprint percentage rate
providers_ranked = providers_group.copy().drop(columns=['all_mdn_orders', 'all_mdn_countries', 'all_mdn_regions',
                                                        'all_mdn_print_time', 'all_reprints_perc'])
providers_ranked['orders_rank'] = providers_ranked['orders_count'].rank(method='min', ascending=False)
providers_ranked['print_time_rank'] = providers_ranked['mdn_print_time'].rank(method='min', ascending=True)
providers_ranked['countries_rank'] = providers_ranked['countries_count'].rank(method='min', ascending=False)
providers_ranked['regions_rank'] = providers_ranked['regions_count'].rank(method='min', ascending=False)
providers_ranked['reprint_perc'] = providers_ranked['reprint_perc']\
                                    .fillna(providers_group['reprint_perc'].median())
providers_ranked['reprints_rank'] = providers_ranked['reprint_perc'].rank(method='min', ascending=True)

In [ ]:
# creating list containing points which providers get for their ranks. 
# Best 20 providers in each category will receive positive points (from 20 to 1)
# Worst 20 will receive negative points (from -1 to -20)
rank_1_20 = [i for i in range(1,21)]
rank_1_20 = rank_1_20[::-1]
rank_21_40 = [0 for i in range(20)]
rank_41_60 = [i*-1 for i in range(1,21)]
rank_points = rank_1_20+rank_21_40+rank_41_60

In [ ]:
# creating function used for points calculation
def calculate_points(provider):
    
    ranks = ['orders_rank', 'print_time_rank', 'countries_rank', 'regions_rank', 'reprints_rank']
    
    points = 0
    
    for rank in ranks:
        data_filtered = providers_ranked.loc[providers_ranked['print_provider_id']==provider]
        data_filtered.reset_index(inplace=True)
        rank_value = data_filtered[rank].at[0].astype(int)

        category_points = rank_points[rank_value-1]
        points += category_points
        
    return points

In [ ]:
# creating df with print providers total points and ranks in each category
providers = providers_ranked['print_provider_id'].to_list()

d = {'print_provider_id': [], 'points': []}
providers_points = pd.DataFrame(data=d)

for provider in providers:
    
    provider_pts = calculate_points(provider)
    providers_points = pd.concat([providers_points, pd.Series({'print_provider_id': provider, 
                                                               'points': provider_pts})\
                                  .to_frame().T], ignore_index=True)
    
providers_points.sort_values(by='points', ascending=False, inplace=True)
providers_points = providers_points.merge(providers_ranked[['print_provider_id', 'orders_rank', 'print_time_rank', 
                                    'countries_rank', 'regions_rank', 'reprints_rank']], on='print_provider_id')

In [ ]:
# providers_points.head()